# [FMA: A Dataset For Music Analysis](https://github.com/mdeff/fma)

Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

## Baselines

* This notebook evaluates standard classifiers from scikit-learn on the provided features.
* Moreover, it evaluates Deep Learning models on both audio and spectrograms.

In [1]:
# Import the library to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'DL project/fma'
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/

!pip install python-dotenv
!pip install --upgrade librosa

import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
import librosa
import librosa.display

%load_ext autoreload
%autoreload 2

import utils


Mounted at /content/drive
/content/drive/My Drive/DL project/fma


In [2]:
# AUDIO_DIR = os.environ.get('AUDIO_DIR')
set_size = 'small' # fma small or medium global variable
AUDIO_DIR = 'data/fma_' + set_size

tracks = utils.load('data/fma_metadata/tracks.csv')
features = utils.load('data/fma_metadata/features.csv')
echonest = utils.load('data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

## Subset

In [3]:
subset = tracks.index[tracks['set', 'subset'] <= set_size]

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks.loc[subset]
features_all = features.loc[subset]

tracks.shape, features_all.shape

Not enough Echonest features: (13129, 767)


((8000, 52), (8000, 518))

In [4]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

6400 training examples, 800 validation examples, 800 testing examples
Top genres (8): ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']
All genres (114): [1, 2, 6, 10, 12, 15, 16, 17, 18, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 38, 41, 42, 45, 46, 47, 49, 53, 58, 64, 66, 70, 71, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 167, 171, 172, 174, 177, 180, 181, 182, 183, 184, 185, 186, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 314, 337, 359, 360, 361, 362, 400, 401, 404, 439, 440, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 695, 741, 763, 808, 811, 1032, 1060, 1193, 1235]


## Making Spectrogram directory

In [5]:
tracks['track', 'genre_top'] = tracks['track', 'genre_top'].astype(str)
labels_onehot = LabelBinarizer().fit_transform(tracks['track', 'genre_top'])
labels_onehot = pd.DataFrame(labels_onehot, index=tracks.index)

In [ ]:
# Just be sure that everything is fine. Multiprocessing is tricky to debug.
utils.FfmpegLoader().load(utils.get_audio_path(AUDIO_DIR, 2))
SampleLoader = utils.build_sample_loader(AUDIO_DIR, labels_onehot, utils.FfmpegLoader())
SampleLoader(train, batch_size=2).__next__()[0].shape

In [ ]:
# loader = utils.FfmpegLoader(sampling_rate=2000)
# out_parent_dir = 'data/spec_' + set_size

# for id in train:
#     in_path = utils.get_audio_path(AUDIO_DIR, id)
#     out_path = utils.get_audio_path(out_parent_dir, id)
#     try:
#         amps = loader.load(in_path).astype(float)
#     except Exception as e:
#         print("\nIgnoring " + in_path +" (error: " + str(e) +").")
#     stft = np.abs(librosa.stft(amps, n_fft=2048, hop_length=8))
#     mel = librosa.feature.melspectrogram(sr=2000, S=stft**2)
#     log_mel = librosa.amplitude_to_db(mel)
#     assert log_mel.shape == (128, 7495)

#     os.makedirs(os.path.dirname(out_path), exist_ok=True)
#     np.save(out_path, log_mel)

#     print(id, end = " ")


In [7]:
def make_spectro_dir(split):
    loader = utils.FfmpegLoaderShort(sampling_rate=2000)
    out_parent_dir = 'data/spec_' + set_size + "2/train"

    for id in split:
        if not id == 139774:
            continue
        class_str = ''.join(labels_onehot.loc[id].astype(str))
        in_path = utils.get_audio_path(AUDIO_DIR, id)
        out_path = os.path.join(out_parent_dir,class_str, str(id))
        try:
            amps = loader.load(in_path).astype(float)
        except Exception as e:
            print("\nIgnoring " + in_path +" (error: " + str(e) +").")
        stft = np.abs(librosa.stft(amps, n_fft=2048, hop_length=512))
        mel = librosa.feature.melspectrogram(sr=22050, S=np.abs(stft)**2)
        log_mel = librosa.power_to_db(mel)

        try:
            assert log_mel.shape == (128, 12)
        except Exception as e:
            print("\nIgnoring " + in_path +" (error: " + str(e) +").")
            continue

        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        np.save(out_path, log_mel)


        print(id, end = " ")
make_spectro_dir(train)

139774 

In [ ]:
# import shutil
# def make_spectro_dir(split):
#     loader = utils.FfmpegLoader(sampling_rate=2000)
#     in_parent_dir = 'data/spec_' + set_size + "/train"
#     out_parent_dir = 'data/spec_' + set_size + "2/train"

#     for id in split:
#         class_str = ''.join(labels_onehot.loc[id].astype(str))
#         in_path = utils.get_audio_path(in_parent_dir, id)
#         out_path = os.path.join(out_parent_dir,class_str, str(id))
#         # try:
#         #     amps = loader.load(in_path).astype(float)
#         # except Exception as e:
#         #     print("\nIgnoring " + in_path +" (error: " + str(e) +").")
#         # stft = np.abs(librosa.stft(amps, n_fft=2048, hop_length=8))
#         # mel = librosa.feature.melspectrogram(sr=2000, S=stft**2)
#         # log_mel = librosa.amplitude_to_db(mel)
#         # assert log_mel.shape == (128, 7495)

#         os.makedirs(os.path.dirname(out_path), exist_ok=True)
#         shutil.move(in_path, out_path)


#         print(id, end = " ")
# # make_spectro_dir(train)
# print(train)


Index([     2,      5,     10,    140,    141,    190,    193,    194,    197,
          200,
       ...
       152570, 153337, 153383, 153452, 153946, 153955, 153956, 154413, 154414,
       155066],
      dtype='int64', name='track_id', length=6400)


In [ ]:
# def make_spectro_dir(split):
#     loader = utils.FfmpegLoader(sampling_rate=2000)
#     out_parent_dir = 'data/raw_' + set_size + "/train"

#     for id in split:
#         in_path = utils.get_audio_path(AUDIO_DIR, id)
#         out_path = utils.get_audio_path(out_parent_dir, id)
#         try:
#             amps = loader.load(in_path).astype(float)
#         except Exception as e:
#             print("\nIgnoring " + in_path +" (error: " + str(e) +").")
#         # stft = np.abs(librosa.stft(amps, n_fft=2048, hop_length=8))
#         # mel = librosa.feature.melspectrogram(sr=2000, S=stft**2)
#         # log_mel = librosa.amplitude_to_db(mel)
#         # assert log_mel.shape == (128, 7495)

#         os.makedirs(os.path.dirname(out_path), exist_ok=True)
#         np.save(out_path, amps)

#         print(id, end = " ")
# make_spectro_dir(train)

In [ ]:
def make_spectro_dir(split):
    loader = utils.FfmpegLoaderShort(sampling_rate=2000)
    out_parent_dir = 'data/raw_' + set_size + "/val"

    for id in split:
        class_str = ''.join(labels_onehot.loc[id].astype(str))
        in_path = utils.get_audio_path(AUDIO_DIR, id)
        out_path = os.path.join(out_parent_dir,class_str, str(id))
        try:
            amps = loader.load(in_path).astype(float)
        except Exception as e:
            print("\nIgnoring " + in_path +" (error: " + str(e) +").")
        # stft = np.abs(librosa.stft(amps, n_fft=2048, hop_length=8))
        # mel = librosa.feature.melspectrogram(sr=2000, S=stft**2)
        # log_mel = librosa.amplitude_to_db(mel)
        # assert log_mel.shape == (128, 7495)

        try:
            amps= amps[:10000]
            assert amps.shape[0] == 10000
        except:
            print(id, " failed- ", amps.shape)

        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        np.save(out_path, amps)

        print(id, end = " ")
make_spectro_dir(val)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-37b8d0cd6c83>", line 28, in <cell line: 28>
    make_spectro_dir(test)
NameError: name 'test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ul

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-37b8d0cd6c83>", line 28, in <cell line: 28>
    make_spectro_dir(test)
NameError: name 'test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactive

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-37b8d0cd6c83>", line 28, in <cell line: 28>
    make_spectro_dir(test)
NameError: name 'test' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactive